# LX 394 / 594 / 694 Homework 3

The time has come to work through the steps we've been discussing in class with respect to the "SHRDLU"-like blocks-world manipulating robot.

Some of this kind of recalls things that we talked about in class, and some of it goes beyond what we did in class.  At the end you'll have an opportunity to extend it a bit too. And it'll be a big benefit for understanding of some of the more complex semantics to have a chance to kind of puzzle through it on your own, even if some of it was kind of breezed through in the several classes that worked through this process.

This is actually super super long.

**Using the "autograder":** This is the same as last time. The way this notebook is set up is that you read a bunch of stuff, and then periodically you will be prompted to answer a question.  The question might take the form of a question you answer in prose, or (more often) a question you answer in code. The code questions come with a test that will check your answer.  If you have it correct, it will tell you that the question "passed."  So, you can be fairly confident as you go along that you got the code working.  You'll want to submit a notebook where things pass.

**Submitting at the end:** When you are ready to submit, download the `.ipynb` file and then go to Gradescope and upload it there.  The autograder will run and double-check your answers.  In principle there may be some tests ("hidden tests") that Gradescope runs that you didn't have access to when you were working through it.

**Getting started**: In order for the checking procedure to work, you need to run this cell below first.  It will download the autograder stuff and the tests.


In [107]:
# Run this first, it makes the autograder go.
# The "%%capture" line below keeps it from showing all the output
%%capture
# Install the autograder (otter-grader) and get the test files
files = "https://github.com/bucomplx/lx394s21/raw/main/assets/ipynb/hw3/tests.zip"
!pip install otter-grader && wget $files && unzip -o tests.zip
# enable the otter test evaluator
import otter
grader = otter.Notebook()

In [2]:
import nltk

# The goals

At the outset, let's consider what we want to get out of this.  We are trying to model a little conversational device that has a domain of expertise (the shapes in a small two-dimensional world) and can converse in a limited way with a human about properties of those blocks, or do things to change the world around.

This is largely about modeling knowledge, but it's also a chance to do some engineering work on how to display the world and get input from a human.  It gives us a chance to construct a grammar that is sophisticated enough to allow us to parse the sentences that we will need.

So, to proceed we need several things, which we will walk through in turn.
- A representation of the objects in the world, and their properties (including their current configuration)
- A way to display the world
- A syntactic grammar that can parse the sentences we want to interpret.
- A semantic representation/grammar that will enable us to determine the truth conditions of a sentence
- An input/output mechanism to allow us to talk to the robot.


# Setting up the model of the world

We will start by defining the model of the world, for this little blocks world.  The concept here is that there is a row of 8 squares that make up the floor, like a chess board, and there a few objects around that have colors and shapes.  The robot is represented by its hand, which can be holding an object or not, and is used to pick things up and put them down.

NLTK has a "semantics" module that can help out with a number of aspects of this interpretation, provided we set up our world in a compatible way.  It is based on a standard way of evaluating semantics and logic from within the philosophical/semantics traditions.  If you wish to read the detailed documentation, it is here: [https://www.nltk.org/api/nltk.sem.html#module-nltk.sem]. The parts we are using are largely in the `logic`, `evaluate`, and `util` portions of the module.

In order to use NLTK's semantics module, we set up a "model".  A model is defined by a "domain of individuals" (which is a list of all the things you can refer to) and a "valuation function", which represents the properties and relationships between the individuals.  The individuals in the domain are represented by short strings (like `"a"`, `"b"`, etc.), and the valuation function is mostly like a Python dictionary.

> In class, we defined the domain of individuals first, and then moved on to define the valuation function.  However, once you have the valuation function, the domain of individuals is somewhat redundant, since you can infer the domain from the valuation function.  So, here we'll just define the valuation function and then infer the domain of individuals afterwards.

Before we get started, let's do a little bit with actual Python dictionaries.  Suppose that we want to define a dictionary that has values for `'dana'`, `'chris'`, `'happy'`, and `'likes'`.  A dictionary is kind of like a list, except instead of referring to the elements by position, you refer to them by name.  The name of an element is known as its "key" and the element named is known as the "value".  Dictionaries are defined using curly braces around the outside, the key to the left of a colon, and the value to the right.  Values can be any kind of object (so, a set, a string, a number, whatever).  So here is our little dictionary that provides values for dana, chris, happy, and likes:

In [3]:
dict_example = {'dana': 'd', 'chris': 'c', 'likes': {('d', 'c')}, 'happy': {'c'}}

If we want to look up who has the property of being happy in this little world, we can use `'happy'` as the key to retrieve the value.

In [4]:
dict_example['happy']

If you iterate over a dictionary, the default behavior is to iterate over the keys, so we can print a list of all the terms that are defined in it by using a `for...in...` iteration:

In [5]:
for entry in dict_example:
  print("Key is: ", entry, " - value for entry is: ", dict_example[entry])

If we wanted to turn that dictionary into an official NLTK Valuation function, we can do this by converting the dictionary into a structure of pairs (where the key is the first element and the value is the second) and then instructing NLTK to build it.

In [6]:
dict_example_tuple_version = list(dict_example.items())
val_example = nltk.Valuation(dict_example_tuple_version)

We can use the Valuation object in much the same way as we used the dictionary we built it from, though it has some extra powers and uses that are provided by NLTK. So we can iterate over this to get the keys and values as well.

In [7]:
for entry in val_example:
  print("Key is: ", entry, " - value for entry is: ", val_example[entry])

One thing to observe is that it has changed the format for "happy" slightly. Generally, an NLTK Valuation function will return either a (string representing a) single individual, or it will return a set of tuples.  For simplicity, NLTK just uses tuples for everything, whether it is a relation requiring a pair (like "likes") or just a property of single objects (like "happy").  So the entity listed as being "happy" there is a 1-tuple of just "c".  Conceptually, our original format (a set of individuals) is probably a bit closer to what we want, but NLTK has made a design decision to minimize the different types a Valuation function can return, and it is easy enough to turn a set of 1-tuples back into a set of individuals.

In [8]:
{x for (x,) in val_example['happy']} # set of the happy individuals

We have a moderately elaborate scenario to describe, with several colors, several shapes, and a relationship between objects.  We could do it just as above, but NLTK provides a slightly easier way to define a Valuation function by writing it out in a string and then parsing it.  We'll use this.

> It is worth noting that we are not required to build a Valuation function from a string.  We built a perfectly good Valuation function above without that.  It's just kind of clunky when the function gets large.  So, we will specify the Valuation function in a string and then ask NLTK to parse that into an actual Valuation function.  However, because we are asking NLTK to do that, we have to play by its rules as far as how the string is formatted. Specifically, it needs to have the keys on the left of a fat arrow (`=>`), sets on the right indicated with curly braces (`{...}`), pairs and tuples on the right indicated with commas and parentheses `(..., ...)`.

Below we define `valstr` as a multiline string (surrounded by triple quotation marks) where each line represents one of the key-value pairings.  The symbol on the left of the fat arrow is an English word (because we are conducting this course in English), but it is not strictly speaking intended to be the language our sentences will eventually be written in.  These are the names of the properties and relationships.  Later in our grammar, we will define the semantic value of the word "pyramid" (an actual English word that might be in a sentence we are parsing) in terms of the symbol `pyramid` that we defined in our Valuation function.

> To attempt to make this distinction a little bit clearer, I have defined the concept of "odd" (referring to the squares in the floor) by the name `oddtastic`.  When we get to the point of defining the grammar, we will define the English word "odd" in terms of the property named `oddtastic`.  The parser will not recognize the word "oddtastic" as being an English word, it will only recognize "odd".  My point in doing this is just to make it somewhat more evident that the words to the left of the fat arrows in the string specification are not supposed to be English, they are a kind of "metalanguage" name for the concept we are defining.

The rest of the specification is probably familiar from repeated review in class, but in short: we have a floor made of blocks arranged so half of them are even and half are odd, we have a ball, two blocks, and two pyramids, each of which has a color, there are two named objects (dana and chris), and they are arranged so that there are three objects on squares and one object atop another.  Object "`e`" is not represented as being on anything because we will later assume that it is intially being held by the robot.

In [9]:
# This is the specification of properties and relationships between individuals
valstr = """
square => {s0, s1, s2, s3, s4, s5, s6, s7}
oddtastic => {s1, s3, s5, s7}
even => {s0, s2, s4, s6}
block => {a, b}
pyramid => {c, e}
ball => {d}
thing => {a, b, c, d, e}
red => {a}
blue => {b, e}
green => {c, d}
on => {(a,s1),(b,s2),(d,s4),(c,d)}
dana => d
chris => c
"""

We can then use this string to create a Valuation function.

In [10]:
val = nltk.sem.Valuation.fromstring(valstr)

In [11]:
# double-check that it worked
print("oddastic: ", val['oddtastic'])
print("dana: ", val['dana'])
print("on: ", val['on'])

When we are defining a model, we need to have both a Valuation function and a domain of individuals.  Now that our Valuation function is defined, we can ask NLTK to just figure out what the domain of individuals is, like so:

In [12]:
print(val.domain)

It's not entirely obvious why, given that the domain of individuals is derivable from the Valuation function, we need to provide both when we ask it to construct a model. NLTK works in mysterious ways.  Anyway, to define the model (which is a machine NLTK can build that is capable of evaluating whether something is true or false within the world it models), we do this:

In [13]:
m = nltk.sem.Model(val.domain, val)

Another thing that we will need in order to actually evaluate whether a logical expression is true or false is to create an "assignment function" which, in effect, tells the interpreter what you are pointing at.  Let me take a quick second to illustrate this.  A basic assignment function in which you are not pointing to anything is created as below.  ("g" is the traditional name for an assignment function.)

> I used `m.domain` instead of `val.domain` below.  Those are just two names for the same set of individuals.  For consistency, I will generally use `m.domain` once the model has been defined. Similarly, I will use `m.valuation` for the valuation function.

In [14]:
g = nltk.Assignment(m.domain)

The arguments for `nltk.Assignment()` are the domain of individuals, and then any pointing functions.  So the reason that the definition of `g` above is an empty pointing is that it has specified the domain of individuals but then no pointing relations.  If we wanted to add a pointing gesture, we can add one as below.  Let's say that our first finger (which we will call `f1`) is pointing to individual `d` (which is the individual we named "dana").  So we add, as a second argument to `nltk.Assignment()`, a list of those pointings.

In [15]:
g2 = nltk.Assignment(m.domain, [('f1', 'd')])

When we are pointing with our first finger at dana, we can evaluate whether dana is green in either of the two ways below. The first is true if dana is green, the second is true if whatever we are pointing at with f1 is green.


In [16]:
expr = nltk.sem.Expression.fromstring("green(dana)")
print(m.satisfy(expr, g))

expr = nltk.sem.Expression.fromstring("green(f1)")
print(m.satisfy(expr, g2))

Incidentally, if we print an assignment function, it will display what you are pointing at (in a pretty unintuitive format, but one that is commonly used in formal semantics). In square brackets after g, it will show an individual from the domain, a slash `/`, and then what is pointing to that individual.

In [17]:
print(g2)

### q1 (is that red thing a block?) ###

**Question:** Now you try it.  Use an assignment function to point to the thing in our model of the world that is red (which you can determine just by looking at the string that specified the valuation function), and evaluate whether it is a block.  

<!--
BEGIN QUESTION
name: q1
-->

In [18]:
# define an assignment function that points to the red thing
g3 = ...
# define expr like above except checking for being a block rather than being green
expr = ...
print("The red thing is a block? ", m.satisfy(expr, g3))

In [ ]:
grader.check("q1")

Ok, back to the thread.  We can anticipate that we are going to be changing this world around eventually, and it would be nice to have a way to reset it back to its original configuration.  So we will at this point define a function called `reset_world()` that will redefine the Valuation function, define the empty assignment function, and also define `obj_in_hand` (what the robot is holding) to be object "e".

In [20]:
def reset_world():
  global valstr, g, m, obj_in_hand
  val = nltk.Valuation.fromstring(valstr)
  m = nltk.Model(val.domain, val)
  g = nltk.Assignment(m.domain)
  obj_in_hand = 'e'

Now, whenever we want to reset things to a known starting configuration, we can just execute `reset_world()`.

> The function above has a line that ensures that the variables being defined are in the global (notebook) space, so that the values are visible to our other functions.  (Had we not done this, some of the values assigned would have only been visible within the function, which would not have been very useful.)

In [21]:
reset_world()

The other thing in this world besides the blocks and floor is the robot.  The robot in this world will be represented by a hand.

At this point, there are some architectural/engineering decisions to make.  I went a particular way in class, but that wasn't forced.  The goal is to represent a robot that can hold things, pick them up, put them down.  In order for us to specify how this robot interacts with the world, we will need to keep track of what the robot is holding.  Since the model has a record of the current configuration (by virtue of having a record of what things are on what other things), it seemed rational to do what I did in class, and consider being held by a robot hand as being part of the current configuration as well.  But it didn't have to be represented that way, and we'll do it differently here just to illustrate that.

There are no laws of physics in this model of the world, no consistency checks to ensure that two objects are not occupying the same space, nothing that ensures that if c is on d, d is not also on c.  We just have a list of properties and relationships.  Because the "on" relationship refers to a physical configuration, we can't translate that into a physical world where things are mutually on one another. But if we had been talking about a "likes" relationship, there might be plenty of examples where two individuals mutually like each other.  The model doesn't know what is a physical relation and what is an emotional one.

All that is to say that we could have kept track of the robot separately from the world, rather than introduce the property "held" into the model.  So, just for variety, we'll make a different architectural choice here, and keep the robot outside the world.  Meaning that model doesn't know what is being held, only the robot will know that.

# Finding the properties of an object

One thing we will need to be able to do is work out what properties an object has.  We at least need to be able to do this in order to know what color and shape to use when drawing an object, but this is also something that we will need to be able to look at when we are trying to resolve reference as well.

As defined, the model knows, for each property, which objects have that property.  What we are trying to do now is determine, for a given object, what properties does it have? It's kind like just turning the lookup table 90 degrees.  

As a reminder, we can walk through the valuation function like we walk through a dictionary.  The code below prints, for each key, what value is associated with that key.

In [22]:
# this will walk through the keys (words) in the valuation function and print
# what value each one refers to.
for v in m.valuation:
  print(v, ": ", m.valuation[v])

Looking over these properties, there are basically three circumstances.  Suppose we are asking about the properties of `c`.  The terms that involve `c` are: a name (`chris`), some one-place properties (`green`, `pyramid`, `thing`), and a relation (`on`).

When we draw the world and when we are resolving referents, we will need to know the one-place properties for color and shape.  We are going to need to determine what is on an object as well, in order to work out the spatial configurations.  But for the moment, let us focus in just on the 1-place properties.  We'll define something more specific in the next section to check the "on" relation.

What we want to do, then, is look at the terms defined by the valuation function and, for any of those that are 1-place predicates, look to see if a particular individual object has the property.  This means we need a way to determine what is and is not a 1-place predicate.  Looking at them, we can see that a 1-place predicate (like "blue") is a set of tuples, where each tuple has a single member.  So, let's just define something that is true of 1-place predicates and not of other things.

In [23]:
def simple_property(p):
  if type(p) == set:
    element = list(p)[0]
    if type(element) == tuple:
      if len(element) == 1:
        return True
  return False

In [24]:
print("Red a 1-place predicate: ", simple_property(m.valuation['red']))
print("On is a 1-place predicate: ", simple_property(m.valuation['on']))
print("Dana is a 1-place predicate: ", simple_property(m.valuation['dana']))

We can then just define the set of terms that includes only the 1-place predicates:

In [25]:
{p for p in val if simple_property(m.valuation[p])}

And then we can define a function that will return any of those 1-place predicates that hold of a given object.

In [26]:
def obj_properties(obj):
  global m
  simple_properties = {p for p in m.valuation if simple_property(m.valuation[p])}
  return {p for p in simple_properties if (obj,) in m.valuation[p]}

Did it work? Let's try it for 'd'.

In [27]:
obj_properties('d')

As an artistic aside, we can make the `simple_property()` function much more compact, by using short-circuit evalution. For fun, you can work out how this does the same thing:

In [28]:
def simpler_property(p):
  return type(p) == set and type(list(p)[0]) == tuple and len(list(p)[0]) == 1

print({p for p in m.valuation if simpler_property(m.valuation[p])})

# Preparing for drawing the world

Beyond knowing what shape and color everything is, we also need to be able to figure out where everything is.  The representation of the world we set up doesn't know that; it only knows what things are on what other things.  We happen to know (by assumption, outside the model) where the squares are in the floor: they are in a line at the bottom.  So in order to figure out where the other objects are, we need to figure out where they are relative to those squares.

We will do this by working our way along the floor, one square at a time, and asking what is on that square (and, if something is on the square, asking what is on that thing, and so on, until we reach the top of the stack of things on a square).

The most basic thing we need to be able to do in order to implement this is to find out, given an object, what (if anything) is on that object.  We need a function that can tell us "what's on" something.  We'll use an assignment function for this, to point at the thing and asking the model "what individuals are on that thing we're pointing at?"

To do this we will use the `satisfiers()` function that the model provides.  To use it, you give it an expression with a variable (like `x`) in it, and you ask "what are the individuals that would lead this expression to be true if we substituted them in for `x`?"

In [29]:
g2 = nltk.Assignment(m.domain, [('f1', 'd')])
expr = nltk.sem.Expression.fromstring("on(x, f1)")
print(list(m.satisfiers(expr, 'x', g2)))

So, that told us that the things that stand in the "on" relation to "d" are: c.  Perfect.  So, let's generalize/package this into a function.  Inside this function we will use `try...except` because NLTK will halt with an error if we ask it what's on a thing when that thing has nothing on it.  We don't want it to stop in that circumstance, we just want it to tell us that we are now at the top of its stack.


In [30]:
def whats_on(obj):
  global m
  # point at the current support and ask: what is on that?
  g2 = nltk.Assignment(m.domain, [('f1', obj)])
  expr = nltk.sem.Expression.fromstring("on(x, f1)")
  try:
    supported_object = list(m.satisfiers(expr, 'x', g2))[0]
  except:
    supported_object = None
  return supported_object

Now, we want to use this to construct a representation of what is stacked on each of the squares.  Minimally, there will be the supporting square at the bottom, but there might be other things on that square.  Square `s4` happens to have a couple of things on it, so let's build that stack by hand, and then we'll make a function that will do it in general.

At the outset, our "stack so far" is empty.  And then we will add the support square.

In [31]:
# at the beginning, nothing in the stack.
stack = []

# we will add the support square next
next_obj = 's4'
stack.append(next_obj)

# now, what is on that thing we just added?
# (that will be the next thing we will want to add to the stack)
next_obj = whats_on(next_obj)
print("The stack is currently: ", stack)
print("The next object for the stack is: ", next_obj)

Notice what we did with `next_obj`.  We added it to the `stack` list, asked `whats_on` it, and then redefined `next_obj` to be whatever that result was.  The idea is that we keep using `next_obj` to mean "the next thing to add to the `stack` list".

The benefit of having done it that way is that we can just repeat the exact thing we just did in order to extend the stack to the next level:

In [32]:
stack.append(next_obj)
next_obj = whats_on(next_obj)
print("The stack is currently: ", stack)
print("The next object for the stack is: ", next_obj)

And then we can repeat the same exact thing again.

In [33]:
stack.append(next_obj)
next_obj = whats_on(next_obj)
print("The stack is currently: ", stack)
print("The next object for the stack is: ", next_obj)

And we've reached the top (which we know because `next_obj` is now `None`).

Now, let's make a function to do this for us.  All it is going to do is just do that same `append`...`whats_on()` step over and over, so long as `next_obj` is not `None`.

### q2 (build_stack) ###

**Question:** Finish up the function below.  Make it do the same thing we just did by hand.  This is borderline trivial if you have been following along. You will add two lines inside the `while` block.  (The lines inside the `while` block will be executed so long as `next_obj` is not `None`, and when `next_obj` is `None` it will stop, and head back out to `return stack`.)

<!--
BEGIN QUESTION
name: q2
-->

In [34]:
def build_stack(square):
  stack = []
  next_obj = square
  while next_obj:
    ...
    next_obj = ...
  return stack

In [ ]:
grader.check("q2")

Success!  So, now that we can build one stack, we can build a stack for each one of the squares.

At this point, let's impose some "physical" constraints on the world.  We're going to line up the squares along the floor in numerical order, so that they represent the horizontal position in this 2-dimensional world.  So, we will define the "floor" as being that list of squares.

In [36]:
floor = ["s{}".format(n) for n in range(8)]
print(floor)

And now we make a list of the stacks on each of the squares of the fllor.

In [37]:
stacks = [build_stack(s) for s in floor]

In [38]:
stacks

And with that, we now have a position for everything, and so we can proceed to draw `stacks` in a slightly more colorful and informative way (though, actually, even just the display above mostly works fine, you just need to tip your head to the side and remember what colors and shapes things are supposed to be).

# Drawing the world

We will use the plotting module `matplotlib` to draw the graphics.  We get this online by running the following commands.  The first one instructs matplotlib to put the graphics here in the document, the `import` command makes Python aware of its commands (and gives us a typing shortcut, we can just refer to `plt` instead of `matplotlib.pyplot`).  The `import Wedge` command makes Python aware of the `Wedge` command we are using to draw the robot hand.

In [39]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge

As for drawing each shape itself and the world as a whole, we kind of talked through it in class, and this function is very similar to the one developed in class.  This version is slightly better in a couple of ways, but we won't dwell on it here.  You can look through it if you want, but we will mostly just use it to draw the world without looking closely at how it works.


In [40]:
def draw_shape(obj, col, row):
  """
  Draw an obj (using its color and shape properties) at col, row
  """
  # define the space in a cell that an object takes up (90% of height and width)
  side = .9
  half = side / 2
  # get the properties of the object
  properties = obj_properties(obj)
  # figure out the color, assume blue if it isn't red or green
  if 'red' in properties:
    color = 'red'
  elif 'green' in properties:
    color = 'green'
  else:
    color = 'blue'
  # draw the right shape
  if 'ball' in properties:
    patch = plt.Circle( (col+half, row+half), half, fc=color)
  elif 'pyramid' in properties:
    patch = plt.Polygon( ((col, row), (col+half, row+side), (col+side, row)), fc=color)
  elif 'square' in properties:
    # squares can be even or not (odd), so make the crosshatch pattern correspond
    if 'even' in properties:
      pattern = '//'
    else:
      pattern = '\\\\'
    patch = plt.Rectangle((col, row), side, side, fc='purple', ec='cyan', hatch=pattern)
  elif 'block' in properties:
    patch = plt.Rectangle((col, row), side, side, fc=color)
  else: # hand
    patch = Wedge((col+1, row + half), half, 30, 330, fc='yellow', ec='black')
  plt.gca().add_patch(patch)
  plt.gca().annotate(obj, (col+half, row+half), va='center', ha='center', color='white', weight='bold', fontsize=16)

Similarly, we'll just define `plot_world()` without much comment.  It was described in class, and this is mostly just the same as it was in class.  It differs a little bit because we are now keeping track of what the robot is holding using the (global) variable `obj_in_hand` rather than making "held" be a property that individuals have in the model.

In [41]:
def plot_world():
  """
  draw the whole blocks world by drawing the stack on each square,
  the held object, and the hand
  """
  global floor, obj_in_hand
  # prepare the canvas
  plt.axes()
  # determine what the piles are on each of the squares
  stacks = [build_stack(s) for s in floor]
  # find the tallest pile (for use in positioning the hand)
  tallest = max( [len(s) for s in stacks] )
  # draw the held object if there is one
  if obj_in_hand:
    draw_shape(obj_in_hand, 1, tallest+1)
  # draw the hand itself (shape designation is None)
  draw_shape(None, 0, tallest + 1)
  # go through the columns
  for col in range(8):
    stack = stacks[col]
    # walk up the stack in this column, drawing each shape
    for row, obj in enumerate(stack):
      draw_shape(obj, col, row)
  # scale the axes so that everything fits on the screen
  plt.axis('scaled')
  # show the final result
  plt.show()

In [42]:
plot_world()

# Affecting the world


The main things we want our robot to do is pick up objects and put them down on other objects.  Before we get into parsing input, let's define some functions that can do these things.
We will start by considering putting an object down.  Since we know what the robot is holding, the information we need is what object to put it on.  What it means to put an object on another one is to take it out of the hand, and add an "on" relation to the valuation function.

We will not do much by way of sanity checking here, since we will do most of that elsewhere.  We will assume that if we use the `put_on()` and `pick_up()` functions we defined below, that we are already sure that, e.g., the robot has something in its hand to put down, that the target is visible, etc.  We will just be doing the picking up and putting down here.

In [43]:
def put_on(target):
  global obj_in_hand, m
  m.valuation['on'] |= {(obj_in_hand, target)}
  obj_in_hand = None

In [44]:
reset_world()
plot_world()
put_on('a')
plot_world()

The other thing we want the robot to do is to pick things up.  Again, no sanity checks here, we will assume that the object to be picked up is already determined to be visible, that the robot isn't already holding anything, etc.  We will just implement the picking up part.

In [45]:
def pick_up(obj):
  global obj_in_hand, m
  m.valuation['on'] = {(x,y) for (x,y) in m.valuation['on'] if x != obj}
  obj_in_hand = obj

In [46]:
reset_world()
put_on('s7')
pick_up('c')
plot_world()

# Building a grammar

Now we will build a grammar that will allow us to parse a small set of sentences about the world.  NLTK provides some tools for this.  If we define a grammar that specifies how words and constituents can be organized into hierarchical tree structures, and how the semantics of each is defined and is combined, then it can largely just do the rest for us. So we will define the grammar.

As a preliminary setup step, let's also make the `svgling` package available, which allows us to be able to draw the tree structures. I will do this without much comment, I did talk through a little of this in class.  It basically installs `svgling`, `import`s it so that Python is aware of the commands it provides, and then defines a function called `convert_tree()` that takes an NLTK-generated tree and turns it into a simpler structure that `svgling` knows how to draw.  In the process the `SEM` (semantic value) features and `CT` (clause type) reatures will be drawn as part of the node label for any node that has those features defined. 

In [47]:
# install tree drawing package svgling
!pip install svgling
import svgling

# define a function to convert a feature tree into something svgling can display
def convert_tree(tree, features=True):
  """
  convert a feature tree into a simple tree that svgling can draw
  """
  nodetype = type(tree).__name__
  if nodetype == 'Tree':
    keylist = list(tree.label().keys())
    label = tree.label()[keylist[0]]
    if features:
      if 'SEM' in tree.label():
        label += "\n" + str(tree.label()['SEM'])
      if 'CT' in tree.label():
        label += "\n" + str(tree.label()['CT'])
    return tuple([label] + [convert_tree(node, features) for node in tree])
  else:
    return tree

There is an engineering/architecture decision made here at the beginning, which I made in class as well.  It is not crucial, and perhaps it is more confusing than useful, but here is the idea:

The grammar is specified with a string that is a lot like the `valstr` string that we used earlier in order to define the Valuation function.  The grammar we will define has a lot of different parts that are largely independent, and I would like to develop the grammar somewhat step by step, and allow for redefining just parts of it while leaving the rest alone.  We could of course just keep modifying a big long grammar-specification string and then using that to define an NLTK grammar, but it seemed a bit more comprehensible to use a Python dictionary to hold the independent parts, and then just assemble the various parts together into a big string right before we use it to create the grammar object.

A quick simple demonstration of the idea:

In [48]:
simple_dictionary = {'first': 'A first bit', 'second': 'Bit number 2', 'prologue': 'Start here'}

In [49]:
print(simple_dictionary.values())

In [50]:
print(sorted(simple_dictionary.values()))

In [51]:
print("\n".join(sorted(simple_dictionary.values())))

That last step creates a string that is built from each of the values in the dictionary, sorted in alphabetical order, and separated by a linebreak (`"\n"`).  You can see that when it is sorted, it goes alphabetically by the value itself (not the alphabetic order of the keys, which would have been `first`, `prologue`, `second`).  The sorting is important because NLTK requires that the first line of a grammar be `% start S` to indicate that the top node of the tree should be the symbol `S`.  It turns out that the character `%` will sort before any alphabetic letters, so by sorting the values we ensure that the `% start` line is right at the top of the string we will build the grammar from.

We will define a function that will do this for us, if we give it a dictionary specifying a grammar.

In [52]:
def assemble_spec(gramspec):
  return "\n".join(sorted(gramspec.values()))

In [53]:
# this is just to show that assemble_spec does what we just did.
print(assemble_spec(simple_dictionary))

Now on to the grammar.  We bring in NLTK's grammar functions, and then start defining parts of the grammar.

In [54]:
from nltk import grammar

gramspec = {}

gramspec['s'] = r"""
% start S
S[SEM=<?subj(?vp)>] -> DP[SEM=?subj] VP[SEM=?vp]
"""

This is defining the very top of the tree, and what it says is: a) the very top node of the tree is `S`, and b) an `S` node is made of a `DP` and `VP`.  That is, a sentence has a subject and a predicate.

The rule looks a bit complex because it covers both the syntactic combination and the rules for combining the semantics. Without the semantics part, the rule is just:

```
S -> DP VP
```

Where the `DP` is the subject of the sentence and the `VP` is the verb phrase.  The `[SEM=...]` parts tacked onto each of the node labels define how the semantics of the subject and VP combine to yield the semantics of the whole sentence.

> Rules of this type are known in theoretical syntax as "rewrite rules" or "phrase structure rules" and the concept is like this: if you have an `S` then you are allowed to rewrite that `S` as `DP` and `VP`.  The trees that we are familiar with from syntax can then be viewed as a record of the applications of these rewrite operations.  We will then have other rules that indicate how a `VP` and `DP` can be rewritten, etc.
>
> In addition to specifying how the syntactic constituency works, it also specifies how the semantics combines.  Every node can have a label and any number of features.  The label is the `S` or `DP` or `VP`.  The features are specfied within square brackets after the label.  Conventionally, the feature's name will be an abbreviation in capital letters, followed by `=`, and then the value that feature has.  If a feature's value starts with `?` then it is a variable, and this has a different meaning on the left side of the arrow and on the right side of the arrow.  On the right side of the arrow, something like `SEM=?vp` means "whatever value the SEM feature has here, call it `?vp`" -- and then on the left side of the arrow, `?vp` means "use whatever value we'd assigned to `?vp` over on the right side of the arrow".  It's like defining a function and naming the arguments; the names are defined on the right side of the arrow, and are used on the left side of the arrow.  So what this `S` rule says with respect to the semantics is that the `SEM` feature of `S` is going to be whatever (function) the `DP` node refers to, applied to whatever (argument) the `VP` node refers to.  The value on the left is enclosed in angled brackets to indicate that it is an expression that needs to be evaluated.

In a way, the syntax and semantics are operating in different directions.  The syntax part is operating top-down.  It says: starting with `S`, rewrite things until we get down to the words that match the sentence.  The semantics starts at the bottom, with the idea that the semantic value of a constituent can be derived from the semantic values of the parts that make it up.  Together, they will provide a hierarchical parse of a sentence that has both syntax and semantics.

Let's keep going so that we can get to a point where we can parse a sentence.  I'm going to just define some stuff here, and we'll come back to why it is defined that way once there is a tree to look at.

Even though we are going to go through it a bit more below, I will just mention here: we are defining two names, a VP that consists of the verb "is" and a PP, a PP that consists of a P and an object, and the preposition "on".

One thing to observe is that we are defining two rules of the form
```
DP -> ...something...
```
These are interpreted as options.  If there is a DP in the structure being derived, then either one of those rules can be applied at that point.  Later on, we will have even more rules with DP on the left side.  But here, one could expand the DP as 'dana' or one could expand the DP as 'chris'.  Either one is legitimate.  The grammar as a whole defines all of the sentences you *can* get to by starting with S and using some combination of rewrite rules until you're down to the words.


In [55]:
# A DP can be rewritten as "dana", in which case the semantics
# should be: "those properties true of dana".  Likewise for "chris".
gramspec['names'] = r"""
DP[SEM=<\P.P(dana)>] -> 'dana'
DP[SEM=<\P.P(chris)>] -> 'chris'
"""

# A VP in this limited grammar is made of a copula (the verb "is")
# and a PP (which will wind up being "on" something).
# The VCOP has no contribution to make to the semantics, the VP
# just has exactly the same semantics that the PP within it has.

gramspec['vp'] = r"""
VP[SEM=?pp] -> VCOP PP[SEM=?pp]
VCOP -> 'is'
"""

gramspec['pp'] = r"""
PP[SEM=<?p(?dp)>] -> P[SEM=?p] DP[SEM=?dp]
"""

# the semantics of "on" here are actually pretty complicated,
# we will work through that in a bit.
gramspec['p'] = r"""
P[SEM=<\X x.X(\y.on(x,y))>] -> 'on'
"""

We have now defined a few fragments of this grammar. We've called those fragments 's', 'names', 'vp', 'pp', and 'p'. Let's assemble them together to see what we have as a whole.

In [56]:
print(assemble_spec(gramspec))

That's just enough to parse "chris is on dana". So, let's do that just to see this in action.

To use this grammar we have specified, we:
- assemble it into a string as above
- use that string to build the grammar
- use the resulting grammar to build a parser
- ask the parser to generate a machine that can parse this specific sentence
- convert this into a list (which is what actually makes the machine run, and parse the sentence)
- take the first parse of the sentence, convert the tree into something `svgling` can draw
- draw it (by letting the value of `draw_tree()` be displayed)

In [57]:
full_spec = assemble_spec(gramspec)
new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
parser = nltk.FeatureChartParser(new_grammar)
words = "chris is on dana".split()
parses = list(parser.parse(words))
svgling.draw_tree(convert_tree(parses[0]))

Now, compare the tree to the grammar. Look at the PP node, it has a `SEM` feature of `\x.on(x,dana)` which is a function that takes an individual as its argument, calls it `x`, and the returns true if that individual we called `x` is on dana.  Put another way, it represents the individuals that are on dana, it is true of all of those individuals and false of any others.

If you look above at the `VP` rule, you can see how it got from the `PP` node to the `VP` node.  To save you from scrolling back up, this was the VP rule:
```
VP[SEM=?pp] -> VCOP PP[SEM=?pp]
```
The `VP` consists of a `VCOP` and ` PP`.  The `VCOP` can be rewritten as 'is'.  The `SEM` feature of the `VP` is just `?pp`, which is whatever the `PP`'s `SEM` feature was.  And in the tree you can see that indeed, the `VP`'s `SEM` feature is that same function, true of individuals that are on dana and false of any others.

To see how it got from `VP` to `S` is a bit trickier. Look at the `S` rule.
```
S[SEM=<?subj(?vp)>] -> DP[SEM=?subj] VP[SEM=?vp]
```
It says that the `SEM` feature of `S` is derived by taking the function represented by the `SEM` feature of the subject (`?subj`) and applying it to the `SEM` feature of the `VP` (`?vp`).  So what is the function that the subject represents?

In the tree above (and in the rule) we can see that it is `\P.P(chris)`. That is, given a predicate, the function returns true if the predicate holds of chris.  It represents all the properties that chris has.  In the `S` rule, this function is applied to the value of the `SEM` feature of `VP`.  The `SEM` feature of `VP` is, again, "on dana"; it is true of any individual that is on dana.  It is the property of being on dana.

So, combining these means that we take the property of being on dana and give it to the function that is true of properties chris has.  Together, they would be saying that the property of being on dana is one of those that chris has.  This will be true if -- and only if -- chris is on dana.

To actually work this out algebraically goes like this: The function `\P.P(chris)` takes a predicate and calls it `P`.  The predicate it is being given is `\x.on(x, dana)`.  So, let's call that predicate `P`, and substitute it in for `P`.  This gives us `\x.on(x, dana)(chris)`.  The function represented by `\x.on(x, dana)` takes an individual and calls it `x`.  The argument "chris" is an individual and is being given to that function.  So, we take chris and name it `x`, then substitute chris in for `x`, yielding: `on(chris, dana)`.  This procedure is sometimes called "lambda conversion".  It's pretty straightforward once you grasp what it is doing.

expression  |  note
-------------------|--
\P.P(chris) ( \x.on(x, dana) )
 | replace P after . with \x.on(x, dana)
\x.on(x, dana)(chris)       | 
 | replace x after . with chris
on(chris, dana) |

Put yet one more way, imagine you have functions.

```python
def subj_chris(fn):
  return fn(chris)

def vp_on_dana(x):
  return((x, 'd') in m.valuation['on'])
```

Here, `vp_on_dana(x)` is a predicate that is true for any `x` you give it that turns out to be on dana.  And `subj_chris(fn)` is a function that takes a function and applies it to chris.  So, combining the two is just `subj_chris(vp_on_dana)`, which will wind up applying `vp_on_dana` to `chris` and be true if chris is on dana.

And this is how we got the `SEM` feature of `S` up at the top to be just `on(chris, dana)`.  It substituted `\x.on(x, dana)` in for `P`, and then substituted chris in for `x`, leading to `on(chris, dana)`.  And that, depending on the configuration of the model at the moment, could either be true or false.

That should I think be good enough to see the basics of how the semantics combines, if it wasn't already clear from class.

Let's look a bit at that definition for "on".

It's a bit hard to see in the abstract how we arrived at this particular formula to represent the semantic value of "on", but it's really kind of just an algebra problem.  We assume that "on" is a function that takes "dana" as an argument.  We know what the `SEM` feature of dana is (it is `\P.P(dana)`), and we know what the combination needs to yield for the `SEM` feature of the `PP` node (`\x.on(x, dana)`).

What it actually says is: `\X.\x.X(\y.on(x,y))`.  So when it combines, we label `\P.P(dana)` as `X` and substitute it in.  This gives us:
```
\x.( \P.P(dana) (\y.on(x,y)))
     ^^^^^^^^^^________________ this was "X"
```

That's hard to unwind, but essentially, we're taking `\y.on(x,y)` to be the argument of `\P.P(dana)` and so we substitute `\y.on(x,y)` in for `P`, resulting in:

```
\x.( \y.on(x,y) (dana))
     ^^^^^^^^^^________________ this was "P"
```

Now, we substitute dana in for `y` and get:

```
\x.on(x,dana)
        ^^^^___________________ this was "y"
```

And that's as simplified as we can make it.  That is in fact the semantics we want, the property of being on dana.


Happily, that's about as complicated as the semantics are going to get.  There is one other thing that's a little bit complicated, and those are the determiners "a" and "every."  We want the robot to be able to parse a sentence like "every block is on a square", which means it needs to understand how to parse "every block".  "Every block" is a DP, it can be a subject of a sentence.  And "block" is a noun, "every" is a determiner.  So we need to work out what each of these pieces need to mean.

In a sense, this is what one studies in Intermediate Semantics, and I can't cram all of that semester into this one project.  I'll try to give a quick rationale anyway.

First, let's handle the nouns.  Those are pretty easy.  The semantics of the noun "block" is something that is true of any individual that is a block.  We have defined what the blocks are in the valuation function.  So, the `SEM` feature for "block" should just be `<\x.block(x)>`, true of anything that the valuation function says is in "block".

We'll just make a bunch of those rules, one per noun.  So that we don't have to type each one in (since they are all the same except for the noun itself), I'll use a little list comprehension to make them.

In [58]:
nouns = ['block', 'ball', 'square', 'pyramid', 'thing']
nounstring = [r"NP[SEM=<\y.{n}(y)>] -> '{n}'".format(n=noun) for noun in nouns]
gramspec['np'] = "\n".join(nounstring) + "\n"

Here are the rules we constructed.

In [59]:
print(gramspec['np'])

While we're here, we can do adjectives too.  The meaning of an adjective is really exactly the same as the meaning of a noun.  "Red" is true of any individual in the "red" set in the valuation function.  So we can define the adjectives the same way.

### q3 (define the adjectives) ###

**Question:** Build the adjectives in the same way we built the nouns.  The adjectives are odd, even, red, green, and blue, and the semantics is the same as for the nouns (so, even should be `<\x.even(x)>`). BUT BEWARE. The English word for odd is "odd" but the definition in the valuation function calls it "oddtastic".  So I have defined that one separately for you.

The autograder will check to make sure that all five adjectives yield a parse.

<!--
BEGIN QUESTION
name: q3
-->

In [60]:
adjs = ['even', 'red', 'green', 'blue'] # skip 'odd', we will add that at the end

adjstrings = ...
...

# add odd (special case: its key in the valuation function is "oddtastic")
gramspec['adj'] = r"Adj[SEM=<\x.oddtastic(x)>] -> 'odd'" + "\n" + gramspec['adj']

In [ ]:
grader.check("q3")

Here are the adjective rules we constructed.

In [62]:
print(gramspec['adj'])

A "DP" (determiner phrase, something that can be a subject or an object) is made of a determiner (like "a" or "every") and a noun phrase (like "square").

So, the basic rule is:
```
DP -> Det NP
```
The determiners we will have are "a" (or "an") and "every".  So, for example, "every block is on dana".  We already know what "block" is (`\x.block(x)`), and we know from before that "is on dana" works out to (`\x.on(x,dana)`).  What "every block is on dana" means is that everything that has the first property (being a block) also has the second property (being on dana).  So, the definition of "every" is something that takes two properties, and is true if having the first property implies having the second property.

This is put into the grammar specification below.  The determiner "a(n)" is also defined there, which I'll review just below it.

In [63]:
gramspec['dp'] = r"""
DP[SEM=<?det(?np)>] -> Det[SEM=?det] NP[SEM=?np]
"""

gramspec['d'] = r"""
Det[SEM=<\N Q.exists x.(N(x) & Q(x))>] -> 'a' | 'an'
Det[SEM=<\N Q.all x.(N(x) -> Q(x))>] -> 'every'
"""

The meaning of "a block is on dana" is somewhat similar to "every block is on dana." It too takes two properties (the property of being a block, and the property of being on dana), and it is true if there is something that has both properties.  If there is something that is a block and also is on dana, then it is true that "a block is on dana."

In [64]:
full_spec = assemble_spec(gramspec)
new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
parser = nltk.FeatureChartParser(new_grammar)
words = "every block is on dana".split()
parses = list(parser.parse(words))
svgling.draw_tree(convert_tree(parses[0]))

Take a moment to look over the tree and see how this works.  You can see that the "every" node has a complicated semantic value that starts with `\N`.  When we combine "every" with "block", the semantic value of "block" gets substituted in for "N", which you can see has happened in the node above it.  This step kind of "peeled away" the \N by substituting in "block", leaving the next level in for the node above it.  The next level in starts with `\Q`, and the semantic value of the VP is substituted in for `Q`, yielding the value at the top: for all individuals "x", if x is a block, then x is on dana.  That is, every block is on dana.

One last thing to look at is how to get adjectives in there.  We defined the semantics of the adjectives, but we don't yet have a rule that puts them into the tree or integrates them with the rest of the semantics.

Imagine that we wanted to say "every green pyramid is on dana".   The adjective "green" goes between the Det and the NP.  Moreover, you can have many adjectives in principle. We can model this by supposing that we attach adjectives to NPs, but the result is an NP.  That way, we can attach another adjective to the NP that resulted from combining an NP and an adjective.  The rule we want is like this:
```
NP -> Adj NP
```
As for the semantics, how is "blue block" related to "block"? Well, the blue blocks are just those blocks that are blue.  Or, the blue blocks are just those blue things that are blocks. Basically, it's just all those things that are both blue and blocks. So the semantics is just a conjunction of the two:

In [65]:
gramspec['adjnp'] = r"""
NP[SEM=<\x.(?adj(x) & ?np(x))>] -> Adj[SEM=?adj] NP[SEM=?np]
"""

In [66]:
full_spec = assemble_spec(gramspec)
new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
parser = nltk.FeatureChartParser(new_grammar)
words = "every blue block is on an odd square".split()
parses = list(parser.parse(words))
svgling.draw_tree(convert_tree(parses[0]))

Complicated, but it looks right.  (And notice that the translation of the English word "odd" uses the symbol "oddtastic", as desired.)  We could even see if it's true. Let's look at the world ourselves and see if it should be true.  Is every blue block on an odd square?

In [67]:
reset_world()
plot_world()

To get the semantics of the top node we ask the tree what its SEM feature is. We can then ask the model whether that formula is true given the current state of the model.

In [68]:
expression = parses[0].label()['SEM']
print(expression)
print(m.satisfy(expression, g))

In [69]:
put_on('s7')
pick_up('b')
put_on('s1')
plot_world()
print(m.satisfy(expression, g))

# Extending beyond declarative clause types

We are already at the point where we could build an interface to the robot that would allow us to type a statement and have the robot tell us whether it is true or not.  But we also want to introduce imperatives (so we can tell the robot to do something) and questions (so we can ask whether something is true).  In class we built the user interface first and then extended it after, but let's just plan ahead now and work out the grammar extensions.

In a sense, we're going to "cheat" for both of these extensions.  That is, we're not going to do a deep syntactic analysis, we're just going to look at the specific types of sentences the robot needs to respond to and detemine how to parse them.

For imperatives, we want to use sentences like "take a green pyramid".  These sentences lack a subject (or have a silent subject).  So, we have a VP made of "take" and an object, and nothing else in the S.  So, we can say something like: "if a sentence has only a VP (that is, if it is missing its subject), then it is an imperative.

For yes-no questions, we are going to rely on the fact that these sentences are all of the "be...on" type, and so forming a yes-no question (like "is a green pyramid on a ball") is accomplished by just reversing the subject and the copula "is".  So, we can say something like: "if a sentence is VCOP DP VP, then it is a yes-no question."  Thinking about it just a bit further, we observe that the VP is itself not a normal VP, it is a VP from which the VCOP has been removed (it was put at the beginning).  Since the VP was just "is" and a PP before, it will in this case look like just a PP.

To put this in the grammar, we just define some additional `S` nodes, for the different configurations.  For yes-no questions it is going to require defining a different VP as well.

Simplistically, we need something like 
```
S -> VCOP DP PP
```
for yes-no questions. This is not quite what we do in syntax classes, but it will do.  As far as the semantics go, we can define the semantics to be the same as they would be for a declarative, given that we will need to check those semantics in order to be able to answer "Yes" or "No".

For imperatives, we need something like
```
# for put a blue block on an even square
S -> V DP PP

# for take a blue block
S -> V DP
```
Again, not that well motivated in terms of theoretical syntax, but it matches what we see.  And it would not be that hard to make this look more like trees you'd make in a Syntax course, but it's not necessary for the present purposes.  For the semantics in imperatives, we won't bother computing the top node's semantics, since it wouldn't be used for anything even if we had computed it.

The main additional thing we need to do is add a feature (`CT` for clause type) on the `S` node so that the parser can see it and react to it.  And add the transitive verb (`VTR`) "take" and the ditransitive verb (`VDT`) "put".

In [70]:
gramspec['s'] = r"""
% start S
S[CT=decl, SEM=<?subj(?vp)>] -> DP[SEM=?subj] VP[SEM=?vp]
S[CT=ynq, SEM=<?subj(?pp)>] -> VCOP DP[SEM=?subj] PP[SEM=?pp]
S[CT=imp] -> VTR DP
S[CT=imp] -> VDT DP PP
VTR -> 'take'
VDT -> 'put'
"""

Here is what we get for a "take" imperative, a "put" imperative, and a yes-no question. For simplicity, I'll omit the features and just display the nodes.

In [71]:
full_spec = assemble_spec(gramspec)
new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
parser = nltk.FeatureChartParser(new_grammar)
words = "take a blue block".split()
parses = list(parser.parse(words))
svgling.draw_tree(convert_tree(parses[0], features=False))

In [72]:
full_spec = assemble_spec(gramspec)
new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
parser = nltk.FeatureChartParser(new_grammar)
words = "put a blue block on an even square".split()
parses = list(parser.parse(words))
svgling.draw_tree(convert_tree(parses[0], features=False))

In [73]:
full_spec = assemble_spec(gramspec)
new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
parser = nltk.FeatureChartParser(new_grammar)
words = "is a blue block on an even square".split()
parses = list(parser.parse(words))
svgling.draw_tree(convert_tree(parses[0], features=False))

# Finding important parts of parses

The interface will get a sentence from the user, parse it, and then react appropriately.  In order to react appropriately it needs to be able to tell what the important properties of the sentence are.  We have defined the grammar to set the `CT` feature of the tree to be the clause type.  We can detect that in a way that is very parallel to how we ask for the `SEM` feature of the tree.


In [74]:
parses = list(parser.parse("is a blue block on an even square".split()))
first_parse = parses[0]
print("Clause type: ", first_parse.label()['CT'])

In [75]:
parses = list(parser.parse("a blue block is on an even square".split()))
first_parse = parses[0]
print("Clause type: ", first_parse.label()['CT'])

In [76]:
parses = list(parser.parse("put a blue block on an even square".split()))
first_parse = parses[0]
print("Clause type: ", first_parse.label()['CT'])

For declaratives and yes-no questions, we can determine whether to say "True" or "Yes" / "False" or "No" by evaluating the `SEM` value of the clause against the model.

In [77]:
parses = list(parser.parse("a blue block is on an odd square".split()))
first_parse = parses[0]
expression = parses[0].label()['SEM']
print('True') if m.satisfy(expression, g) else print('False')

In [78]:
parses = list(parser.parse("is a blue block on an odd square".split()))
first_parse = parses[0]
expression = parses[0].label()['SEM']
print('Yes') if m.satisfy(expression, g) else print('No')

For imperatives, we need to determine what the verb is, what the object is, and -- if the verb is ditransitive -- where the object will go.  As a reminder, here is what we have for "take a block".

In [79]:
full_spec = assemble_spec(gramspec)
new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
parser = nltk.FeatureChartParser(new_grammar)
words = "take a block".split()
parses = list(parser.parse(words))
svgling.draw_tree(convert_tree(parses[0], features=False))

The verb is going to be the first daughter of `S`, so we can treat the tree as a list and look for the first element.  The first element is the nonterminal node (`VTR` or `VDT`) and the first element of *that* is the word itself.

In [80]:
parses = list(parser.parse("take a blue block".split()))
first_parse = parses[0]
print('Verb nonterminal node is: ', first_parse[0])
print('Verb is: ', first_parse[0][0])

In [81]:
parses = list(parser.parse("put a blue block on an even square".split()))
first_parse = parses[0]
print('Verb nonterminal node is: ', first_parse[0])
print('Verb is: ', first_parse[0][0])

The object is going to be the second element, for both types of verb.  This will have a `SEM` feature associated with it that will allow us to determine what it refers to.

In [82]:
parses = list(parser.parse("put a blue block on an even square".split()))
first_parse = parses[0]
print("Object's SEM feature is: ", first_parse[1].label()['SEM'])

What are the blue blocks? We can ask the model what the options are by asking it what values of `y` make the statement "`y` is a blue block" true.  Below, we use NLTK's semantics module to create a predicate `is_y` that is true of things that are y, we give that to the expression we got from the object's `SEM` feature, and then we use the model's `satisfiers()` function to determine which values of `y` make "`y` is a blue block" true.

In [83]:
parses = list(parser.parse("put a blue block on an even square".split()))
first_parse = parses[0]
obj_sem = first_parse[1].label()['SEM']
is_y = nltk.sem.Expression.fromstring(r"\x.(x=y)")
obj_is_y = nltk.sem.ApplicationExpression(obj_sem, is_y).simplify()
options = m.satisfiers(obj_is_y, 'y', g)
print("Options: ", options)

The last thing we need to be able to figure out is where the object is to be put for the verb "put". To do this, we need to look at what the object of "on" is. Let's draw a tree, so we can trace through it visually.


In [84]:
parses = list(parser.parse("put chris on a square".split()))
svgling.draw_tree(convert_tree(parses[0], features=False))

Looking at the tree, we can see that the `PP` is the third daughter of `S`, and the object is the second daughter of the `PP`.  So:

In [85]:
parses = list(parser.parse("put a blue block on an even square".split()))
first_parse = parses[0]
print("Target for put has a SEM feature of: ", first_parse[2][1].label()['SEM'])

Let us consolidate the conclusions above into functions that we can use in the interface.

In [86]:
def parse_sentence(gramspec, sentence):
  full_spec = assemble_spec(gramspec)
  new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
  parser = nltk.FeatureChartParser(new_grammar)
  words = sentence.split()
  parses = list(parser.parse(words))
  return parses

def clause_type(parse):
  return parse.label()['CT']

def clause_sem(parse):
  return parse.label()['SEM']

def object_sem(parse):
  return parse[1].label()['SEM']

def dp_options(dp_sem):
  is_y = nltk.sem.Expression.fromstring(r"\x.(x=y)")
  dp_is_y = nltk.sem.ApplicationExpression(dp_sem, is_y).simplify()
  options = m.satisfiers(dp_is_y, 'y', g)
  return options

def imp_verb(parse):
  return parse[0][0]

def check_truth(parse):
  global m
  g = nltk.Assignment(m.domain)
  return m.satisfy(clause_sem(parse), g)

### q4 (interpreting the target of put) ###

**Question:** Add one more function called `target_sem(parse)` to those above that will find the semantic value for the target of "put".  For example, if the sentence were "put a blue block on an even square", as above, we want the `target_sem(parse)` function to return the `SEM` feature of the DP "an even square".  This is mostly just to ensure that you are following along.  You can model this on `object_sem(parse)` above, finding the node in the way we just did immediately above.

<!--
BEGIN QUESTION
name: q4
-->

In [87]:
def target_sem(parse):
  ...

In [ ]:
grader.check("q4")

Just to reassure ourselves that we have replicated what we did by hand:

In [89]:
parses = parse_sentence(gramspec, "put a blue block on an even square")
first_parse = parses[0]
print("Clause type: ", clause_type(first_parse))
print("Verb (for imperative): ", imp_verb(first_parse))
print("Object SEM: ", object_sem(first_parse))
print("Object options: ", dp_options(object_sem(first_parse)))
print("Target SEM: ", target_sem(first_parse))
print("Target options: ", dp_options(target_sem(first_parse)))
parses = parse_sentence(gramspec, "dana is on a blue block")
first_parse = parses[0]
print("Clause SEM: ", clause_sem(first_parse))
print("This holds in the model? ", check_truth(first_parse))

We now have assembled enough tools that we can write up a user interface to the robot pretty simply.

# Talking to the robot

Now we will write a short function that will collect a sentence from the user, parse it, and act on it.  In class we designed a little chat program that would operate in a loop, but here we'll define a function called `hey_robot()` that will take the sentence and act on it.

> For fun, I've also included a definition of `chat()` that makes use of `hey_robot()` so that if you want to you can have what feels like a more interactive session with the robot.  In the `chat()` function, you type `bye` to quit.

In `hey_robot()` there are two special "magic commands", one being `look` to show the world, and a third being `reset` to reset the world to its original state.

In order to divide up the labor and make the architecture of the system flexible, we will define separate functions for the main input processor, and handlers for each of the clause types.

When we collect the input from the user, we will force it to be lowercase and remove any `?` characters that the user might have added (i.e. to a yes-no question).

In [90]:
def hey_robot(sentence):
  sentence = sentence.lower().replace("?", "")
  if sentence == 'look':
    plot_world()
  elif sentence == 'reset':
    reset_world()
    print('World reset to initial state.')
  else:
    try:
      parses = parse_sentence(gramspec, sentence)
      first_parse = parses[0]
      ct = clause_type(first_parse)
      if ct == 'ynq':
        do_ynq(first_parse)
      elif ct == 'imp':
        do_imp(first_parse)
      else: # decl
        do_decl(first_parse)
    except Exception as e:
      print("Error: {}".format(e))
      print("Sorry, what?")

In [91]:
def chat():
  print("Type 'bye' to leave, 'look' to show the scene, 'reset' to reset the scene.")
  print()
  sentence = 'look'
  while sentence != 'bye':
    hey_robot(sentence)
    sentence = input("> ")
  print("Bye!")

The `hey_robot()` function above relies on functions `do_decl()`, `do_ynq()`, and `do_imp()`, but they have not been defined yet.  We can do the declarative and yes-no question ones pretty easily, those both just check whether the sentence is true and then print something appropriate.

The `do_imp()` one is more complicated, so we'll for the moment just define it to say "Out of order."

In [92]:
def do_decl(parse):
  if check_truth(parse):
    print('True')
  else:
    print('False')

def do_ynq(parse):
  if check_truth(parse):
    print('Yes')
  else:
    print('No')

def do_imp(parse):
  print("Out of order.")

That is enough to talk to the robot initially, get it to say whether statements about the world are true or false, answer yes or no, and show or reset the world.  We left `do_imp()` defined to do nothing substantive yet, since we'll turn to that next.

In [93]:
hey_robot('reset')

In [94]:
hey_robot('a pyramid is on a ball')

In [95]:
hey_robot('is a blue thing on a square?')

For the imperatives, there are two different distinct things it can do: `take` and `put`.  We defined a couple of functions long ago that would pick things up and put them on things, but we'll incorporate what those functions did into what is happening here.  (Mostly, that is because now we need to contemplate the possibility that there are multiple options.)

If we are trying to pick up an object, we can use `dp_options()` to determine what objects in the world match the description.  But not all objects in the world are takable.  Some objects are elements of the floor, and some objects are hidden under other objects. However, so long as there is some object that meets the description and is takable, we should take that object.  There might be multiple things we could take, we'll just pick one if so.

If we are trying to put an object on another object, this is a compound action.  It requires picking up the object first and then putting it down.  Which also means that if we are already holding an object, we need to put that object down first.  The world has been strategically designed so that there are more squares in the floor than there are objects, so it is guaranteed that there will always be some square somewhere that is empty, so we can put something on an empty square and it will not obscure anything we might need to leave visible.  So, let's define something that will give us one of those empty squares.


In [96]:
def empty_square():
  for square in floor:
    if not whats_on(square):
      return square

We also need to know what the visible objects are.  These are the things that have nothing on top of them. So, we'll define a function that will give us the visible objects. We also want to exclude the object that is in the hand, since that can't be taken or used as a target for putting something on.

In [97]:
def visible_things():
  global m, obj_in_hand
  return {obj for obj in m.domain - {obj_in_hand} if not whats_on(obj)}

The visible objects are places we can put something. This is not quite the same as takable objects, because the squares of the floor count as visible objects but are not takable.  So, let's define the takable things too.

In [98]:
def takable_things():
  global floor
  return visible_things() - set(floor)

In [99]:
print("Visible things: ", visible_things())
print("Takable things: ", takable_things())

And that's pretty much all we need to implement our imperative function.  We will split this up into a `do_put()` and a `do_take()` function because we actually need to do a `do_take()` as part of both actions.

In [100]:
def do_imp(parse):
  global obj_in_hand, floor
  verb = imp_verb(parse)
  obj_options = dp_options(object_sem(parse))
  if verb == 'take':
    if do_take(obj_options):
      print('Taken.')
  elif verb == 'put':
    if do_put(obj_options, parse):
      print('Placed.')
  else:
    print("I don't understand what you are asking me to do.")

In [101]:
def do_take(obj_options):
  global obj_in_hand, floor
  if len(obj_options) == 0:
    print("There is no such thing to take.")
  else:
    if obj_in_hand in obj_options:
      print("Already holding a thing of the right kind.")
      return True
    else:
      if len(obj_options - set(floor)) == 0:
        print("The floor is bolted down.")
      else:
        takable = obj_options & takable_things()
        if len(takable) == 0:
          print("I cannot see such a thing to take.")
        else:
          if obj_in_hand:
            put_on(empty_square())
          obj_to_take = list(takable)[0]
          pick_up(obj_to_take)
          return True
  # unless we hit one of the success conditions, report that we failed.
  return False

In [102]:
def do_put(obj_options, parse):
  if do_take(obj_options):
    target_options = dp_options(target_sem(parse))
    if len(target_options) == 0:
      print("There is no such place to put anything.")
    else:
      visible = target_options & visible_things()
      if len(visible) == 0:
        print("I cannot see such a place to put anything.")
      else:
        target = list(visible)[0]
        put_on(target)
        return True
  return False

At this point, you can play around with the robot a bit if you wish.  Below I've tried a couple of things that are of interest, and I've left some blank cells for your further entertainment as well.

In [103]:
hey_robot('reset')
hey_robot('put a red block on a blue pyramid')
hey_robot('look')

In [104]:
hey_robot('take an odd square')

In [105]:
hey_robot('take a blue pyramid')

In [106]:
hey_robot('take a pyramid')
hey_robot('look')

In [157]:
hey_robot('take a blue thing')
hey_robot('look')

# Pyramids are pointy

We've got a pretty sophisticated system now, but there are lots of further directions you could take this if you desired.  For example, you might:

- add "under"
- add containers, object sizes, "in", "empty", "full"
- make the world three dimensional and add predicates like "beside" and "north of", "east of".
- add support for wh-questions so you can ask "what is on a blue block?" or "how many green things are on an even square?" or "where is a red block?"
- add support for "the" so that it will assume that if you were previously talking about a green pyramid, subsequent reference to "the pyramid" will refer to the green pyramid. This requires implementing at least a limited amount of memory of the preceding discourse.
- add memory of name assignments and a way to name things so you can, e.g. "call the red block andy" and then refer to it as "andy" for the rest of the conversation
- add planning support so that if you try to pick up something that is not visible, the robot will first clear off the surface to make it visible, and then pick it up.
- add support for multi-step actions so you can "put every blue thing on an even square"

I will ask you to make just one extension here, to help convince yourself that you kind of understand how this works. The goal is going to be to prevent the robot from putting something on a pyramid.  Because pyramids are pointy, the thing would just fall off.

> Even this could be moderately sophisticated.  The simple version is just to prevent putting anything on a pyramid altogether.  But you could also, for example, permit something on a pyramid just in case it is next to an object it could lean on.  Though in that case, probably the object should fall off into a neighboring column if the support is removed. Etc.  To begin with, just prevent anything from being placed on a pyramid at all, in order to satisfy the autograder.  If you would like to elaborate afterwards, feel free!

### q5 (you can't put something on a pyramid) ###

**Question:** This one is pretty free form.  Make some modifications so that when we try to get the robot to put an object on a pyramid it will refuse to do so.  Make this happen in whatever way you'd like; the autograder will try to put something on a pyramid and then check to be sure that the world has not changed.

<!--
BEGIN QUESTION
name: q5
-->


In [158]:
# Put whatever function redefinitions you need to put in here.
...

In [ ]:
grader.check("q5")

## Submission instructions ##

Go to File at the upper left of this web page and click "Download .ipynb" to download a copy of this.  Then go to Gradescope to submit the homework, and drag the .ipynb file in.  That should be all you need to do.  The autograder will run, but if your tests all passed in here, they should pass there as well.

There are no other tests here, but if you want to add stuff to the end of the notebook to implement other things, feel free.